In [1]:
import requests
from requests.auth import HTTPBasicAuth
import os
from dotenv import load_dotenv
import pandas as pd
import datetime

def convert_unix_timestamp(timestamp):
    dt = datetime.datetime.utcfromtimestamp(timestamp)
    return dt.strftime('%m/%d/%Y')

# Carregue as variáveis de ambiente a partir do arquivo .env
load_dotenv()
username = os.getenv("ELOQUA_USERNAME")
password = os.getenv("ELOQUA_PASSWORD")

base_url = "https://secure.p04.eloqua.com"  # Substitua YOUR_ELOQUA_INSTANCE pelo nome da sua instância do Eloqua

# Autenticação usando nome de usuário e senha
auth = HTTPBasicAuth(username, password)

# Inicialize uma lista para armazenar os dados
data_list = []

page = 1

response = requests.get(f"{base_url}/api/REST/1.0/assets/optionLists?page={page}&depth%20=partial", auth=auth)
if response.status_code == 200:
    response_json = response.json()
    total_pages = (response_json.get("total") // response_json.get("pageSize")) + 1

    while page <= total_pages:
        response = requests.get(f"{base_url}/api/REST/1.0/assets/optionLists?page={page}&depth%20=partial", auth=auth)

        if response.status_code == 200:
            # Analise o JSON da resposta
            response_json = response.json()
            for item in response_json['elements']:
                id = item.get('id', None)
                name = item.get('name', None)
                createdAt = item.get('createdAt', None)
                if createdAt is not None:
                    createdAt = convert_unix_timestamp(int(createdAt))  # Converta para número 
                data_list.append([id, name, createdAt])

            page += 1
        else:
            print("Falha na solicitação à API.")
            break

    # Crie um DataFrame pandas com os dados
    df = pd.DataFrame(data_list, columns=['id', 'name', 'createdAt'])
    df.to_excel('picklist.xlsx', index=False)

    print("Planilha 'picklist.xlsx' criada com sucesso!")
else:
    print("Falha na solicitação à API.")


Planilha 'picklist.xlsx' criada com sucesso!


: 